<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setting" data-toc-modified-id="Setting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setting</a></span></li><li><span><a href="#Function" data-toc-modified-id="Function-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Function</a></span></li></ul></div>

In [6]:
import sys
import os
import time
import datetime
import pickle
import json
import re
import operator
from collections import defaultdict

import pprint as pp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


sys.path.append("/Users/xuzhu/Desktop/code/assistants") # my package
from toolbox.os_assistant import scan_folder

---

## Setting

In [7]:
DATA_FOLDER = "/Users/xuzhu/Desktop/data/open_dataset"

In [32]:
posting_data__temp = [
    "my dog has flea problems help please",
    "maybe not take hime to dog park stupid",
    "my dalmation is so cute I love him",
    "stop posting stupid worthless garbage",
    "mr licks ate my steak how to stop him",
    "quit buying worthless dog food stupid"
]

label__temp = [0, 1, 0, 1, 0, 1]
posting_data__dict = {"posting_content": posting_data__temp}
posting_data__df = pd.DataFrame(posting_data__dict)
posting_data__df["insult"] = label__temp
posting_data__df

,posting_content,insult
0,my dog has flea problems help please,0
1,maybe not take hime to dog park stupid,1
2,my dalmation is so cute I love him,0
3,stop posting stupid worthless garbage,1
4,mr licks ate my steak how to stop him,0
5,quit buying worthless dog food stupid,1


In [68]:
list(posting_data__df.values)

data__list = [list(i) for i in list(posting_data__df[["posting_content"]].values)]
data__list

list(posting_data__df["posting_content"].values)

[array(['my dog has flea problems help please', 0], dtype=object),
 array(['maybe not take hime to dog park stupid', 1], dtype=object),
 array(['my dalmation is so cute I love him', 0], dtype=object),
 array(['stop posting stupid worthless garbage', 1], dtype=object),
 array(['mr licks ate my steak how to stop him', 0], dtype=object),
 array(['quit buying worthless dog food stupid', 1], dtype=object)]

[['my dog has flea problems help please'],
 ['maybe not take hime to dog park stupid'],
 ['my dalmation is so cute I love him'],
 ['stop posting stupid worthless garbage'],
 ['mr licks ate my steak how to stop him'],
 ['quit buying worthless dog food stupid']]

['my dog has flea problems help please',
 'maybe not take hime to dog park stupid',
 'my dalmation is so cute I love him',
 'stop posting stupid worthless garbage',
 'mr licks ate my steak how to stop him',
 'quit buying worthless dog food stupid']

## Function

In [77]:
def create_uniq_vocab_list(
    df,
    column
):  
    statements = list(df[column].values)
    vocabulary_list = [s.strip().split(" ") for s in statements] # [[w1, w2, w3], [w1, w2, w3],...]
    
    vocab_set = set()
    for row in vocabulary_list:
        vocab_set = vocab_set | set(row)
            
    uniq_vocab_list = list(vocab_set)
    return uniq_vocab_list


label_list = posting_data__df.iloc[:,-1].to_list()
posting_vocab = create_uniq_vocab_list(
    df=posting_data__df,
    column="posting_content"
)

print(
    posting_vocab[:2],
    len(posting_vocab)
)
label_list

['I', 'maybe'] 33


[0, 1, 0, 1, 0, 1]

In [92]:
def create_word_vector(
    uniq_vocab_pool,
    input_words # uniq set
    
):
    results = {}
    results["word_vector"] = [0] * len(uniq_vocab_pool)
    results["new_words"] = []
    
    if input_words is set:
        pass
    else:
        input_words = set(input_words)
        
    for word in input_words:
        if word in uniq_vocab_pool:
            matched_index = uniq_vocab_pool.index(word)
            results["word_vector"][matched_index] = 1
        else:
            print("The word '{0}' is not in my vocabulary list".format(word))
            results["new_words"].append(word)
            
    return results
        

input__test = set(["a", "stupid", "dog", "problems", "please", "b"])

match_results = create_word_vector(
    uniq_vocab_pool=posting_vocab,
    input_words=input__test
)


match_results.keys()
input__word_vector = match_results["word_vector"]
input__new_words = match_results["new_words"]

input__word_vector[:15]
input__new_words

The word 'b' is not in my vocabulary list
The word 'a' is not in my vocabulary list


dict_keys(['word_vector', 'new_words'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

['b', 'a']

In [91]:
posting_data__df

positing_data__label = posting_data__df.iloc[:,-1].to_list()
positing_data__uniq_vocab_pool = create_uniq_vocab_list(df=posting_data__df, column="posting_content")

,posting_content,insult
0,my dog has flea problems help please,0
1,maybe not take hime to dog park stupid,1
2,my dalmation is so cute I love him,0
3,stop posting stupid worthless garbage,1
4,mr licks ate my steak how to stop him,0
5,quit buying worthless dog food stupid,1


In [130]:
print(positing_data__uniq_vocab_pool)

['I', 'maybe', 'flea', 'ate', 'mr', 'help', 'love', 'has', 'take', 'buying', 'my', 'worthless', 'stop', 'garbage', 'problems', 'not', 'please', 'so', 'posting', 'food', 'steak', 'park', 'licks', 'cute', 'him', 'hime', 'dalmation', 'stupid', 'how', 'quit', 'is', 'dog', 'to']


In [101]:
training_data__matrix = []

for index, row in posting_data__df.iterrows():
    word_list = row["posting_content"].strip().split(" ")
    matched_results = create_word_vector(
        uniq_vocab_pool=positing_data__uniq_vocab_pool,
        input_words=word_list
    )
    training_data__matrix.append(matched_results["word_vector"])


# [word_vector, word_vector,...]
type(training_data__matrix[0])

training_data__dict = {
    "vector": training_data__matrix,
    "label": positing_data__label
}
training_data__df = pd.DataFrame(data=training_data__dict)
training_data__df

list

,vector,label
0,"[0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, ...",0
1,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",1
2,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...",0
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, ...",1
4,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...",0
5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",1


In [134]:
sum(positing_data__label)

(np.zeros(5) + [1, 0, 1, 0, 1]) / 3

3

array([0.33333333, 0.        , 0.33333333, 0.        , 0.33333333])

$$p({c_i}|\vec{w}) = \frac{p(\vec{w} c_i)}{p(\vec{w})}=\frac{p(\vec{w}|c_i) \cdot p(c_i)}{p(\vec{w})}$$

Assume: $p(\vec{w}|c_i) = p(w_0, w_1, \cdots | c_i) = p(w_0|c_i) \cdot p(w_1|c_i) \cdot ...$

In [135]:
def train_naive_bayesian(
    training_data__matrix,
    training_data__label
):
    """
    Here is the description
    """
    
    statement_qty = len(training_data__label)
    uniq_vocab_qty = len(training_data__matrix[0]) # uniq vocabulary qty
    prob__label_1 = sum(training_data__label) / statement_qty # p(c), here label = (0, 1) only 2 values
    
    label_0__vocab_array, label_1__vocab_array = np.zeros(uniq_vocab_qty), np.zeros(uniq_vocab_qty)
    label_0__vocab_sum, label_1__vocab_sum = 0, 0
    
    for i in range(statement_qty):
        if training_data__label[i] == 1: # label 1 statement
            label_1__vocab_array += training_data__matrix[i]
            label_1__vocab_sum += sum(training_data__matrix[i])
        else: # label 0 statement
            label_0__vocab_array += training_data__matrix[i]
            label_0__vocab_sum += sum(training_data__matrix[i])
    
    label_1__vector = label_1__vocab_array / label_1__vocab_sum
    label_0__vector = label_0__vocab_array / label_0__vocab_sum
            
    return label_0__vector, label_1__vector, prob__label_1
    

In [136]:
p0v, p1v, p1b = train_naive_bayesian(training_data__matrix, positing_data__label)

In [139]:
p0v, p1v, p1b

(array([0.04166667, 0.        , 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.        , 0.        ,
        0.125     , 0.        , 0.04166667, 0.        , 0.04166667,
        0.        , 0.04166667, 0.04166667, 0.        , 0.        ,
        0.04166667, 0.        , 0.04166667, 0.04166667, 0.08333333,
        0.        , 0.04166667, 0.        , 0.04166667, 0.        ,
        0.04166667, 0.04166667, 0.04166667]),
 array([0.        , 0.05263158, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.05263158, 0.05263158,
        0.        , 0.10526316, 0.05263158, 0.05263158, 0.        ,
        0.05263158, 0.        , 0.        , 0.05263158, 0.05263158,
        0.        , 0.05263158, 0.        , 0.        , 0.        ,
        0.05263158, 0.        , 0.15789474, 0.        , 0.05263158,
        0.        , 0.10526316, 0.05263158]),
 0.5)